In [1]:
MAX_WEIGHT = 250
CHROMOSOME_SIZE = 12
BOXES = {
    1: (20, 6),
    2: (30, 5),
    3: (60, 8),
    4: (90, 7),
    5: (50, 6),
    6: (70, 9),
    7: (30, 4),
    8: (30, 5),
    9: (70, 4),
    10: (20, 9),
    11: (20, 2),
    12: (60, 1)
}

In [2]:
def fitness(chromosome):
    score = 0
    for i, c in enumerate(chromosome):
        weight, _ = BOXES[i+1]
        score += weight*c
    if score > MAX_WEIGHT:
        f = 0
    else:
        score = 0
        for i, c in enumerate(chromosome):
            _, cost = BOXES[i+1]
            score += cost*c
        f = score
    return f

In [3]:
test_c = [0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1]
fitness(test_c)

0

In [4]:
import random
import numpy as np 

def generate_chromosome():
    chromosome = np.zeros(CHROMOSOME_SIZE)
    for i in range(len(chromosome)):
        if random.uniform(0,1) < .5:
            chromosome[i] = 1
        else:
            chromosome[i] = 0
    
    score = 0
    for i, c in enumerate(chromosome):
        weight, _ = BOXES[i+1]
        score += weight*c
    if score > MAX_WEIGHT:
        generate_chromosome() # create a new one if it exceeds the max weight
    return chromosome

In [5]:
fitness(generate_chromosome())

0

In [6]:
def generate_population(population_size):
    population = []
    for i in range(population_size):
        chromosome = generate_chromosome()
        population.append(chromosome)
    return np.array(population)

In [7]:
ex_pop = generate_population(1000)
ex_pop

array([[0., 1., 1., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 1., 0., 0.]], shape=(1000, 12))

In [8]:
def evaluate(population):
    fitnesses = np.zeros(population.shape[0])
    for i in range(population.shape[0]):
        f = fitness(population[i])
        fitnesses[i] = f
    max_id = fitnesses.argmax()
    return population[max_id]

In [9]:
def evaluate_and_select(population):
    fitnesses = np.zeros(population.shape[0])
    for i in range(population.shape[0]):
        f = fitness(population[i])
        fitnesses[i] = f
    
    parent1 = random.choices(population, weights=fitnesses, k=1)[0]
    parent2 = random.choices(population, weights=fitnesses, k=1)[0]
    return parent1, parent2

In [10]:
parent1, parent2 = evaluate_and_select(ex_pop)
parent1, parent2

(array([1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1.]),
 array([1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0.]))

In [11]:
best = evaluate(ex_pop)
best

array([1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0.])

In [12]:
def crossover(parent1, parent2):
    crossover_point = random.randint(0, CHROMOSOME_SIZE-1)
    child1 = np.hstack([parent1[:crossover_point], parent2[crossover_point:]])
    child2 = np.hstack([parent2[:crossover_point], parent1[crossover_point:]])
    return child1, child2

def mutate(chromosome):
    mutation_point = random.randint(0, CHROMOSOME_SIZE-1)
    if chromosome[mutation_point] == 0:
        chromosome[mutation_point] = 1
    else:
        chromosome[mutation_point] = 0
    return chromosome

In [13]:
GENERATIONS = 15
POP_SIZE = 100
MUTATION_PROB = .2

population = generate_population(POP_SIZE)

for _ in range(GENERATIONS):
    parent1, parent2 = evaluate_and_select(population)
    child1, child2 = crossover(parent1, parent2)

    if random.uniform(0,1) < MUTATION_PROB:
        child1 = mutate(child1)
    if random.uniform(0,1) < MUTATION_PROB:
        child2 = mutate(child2)
    
    population = np.vstack([np.array([child1, child2]), population[2:]])

best = evaluate(population)

In [14]:
best

array([1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0.])

In [17]:
total_weight = 0
total_importance = 0
for i, b in enumerate(best):
    weight, importance = BOXES[i+1]
    total_weight += b*weight
    total_importance += b*importance

In [18]:
total_weight, total_importance

(np.float64(230.0), np.float64(40.0))

In [1]:
from genetic import Genetic

g = Genetic(100, 1000, .3)
g.algorithm()

Object 1 included, weight: 20, importance: 6
Object 2 included, weight: 30, importance: 5
Object 3 included, weight: 60, importance: 8
Object 6 included, weight: 70, importance: 9
Object 8 included, weight: 30, importance: 5
Object 10 included, weight: 20, importance: 9
Object 11 included, weight: 20, importance: 2


(np.float64(250.0),
 np.float64(44.0),
 array([1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0.]))

In [2]:
gens = [10, 20, 50, 100, 200]
pops = [10, 20, 50, 100, 200]
muts = [.1, .2, .3, .4, .5]

results = {}
for gen in gens:
    for pop in pops:
        for mut in muts:
            g = Genetic(gen, pop, mut)
            weight, importance, chromosome = g.algorithm(print_results=False)
            results[(gen, pop, mut)] = (weight, importance, chromosome)

In [3]:
results

{(10, 10, 0.1): (np.float64(240.0),
  np.float64(28.0),
  array([1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1.])),
 (10, 10, 0.2): (np.float64(230.0),
  np.float64(40.0),
  array([1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0.])),
 (10, 10, 0.3): (np.float64(250.0),
  np.float64(35.0),
  array([0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0.])),
 (10, 10, 0.4): (np.float64(240.0),
  np.float64(36.0),
  array([1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0.])),
 (10, 10, 0.5): (np.float64(250.0),
  np.float64(43.0),
  array([1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0.])),
 (10, 20, 0.1): (np.float64(250.0),
  np.float64(36.0),
  array([1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1.])),
 (10, 20, 0.2): (np.float64(240.0),
  np.float64(42.0),
  array([1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0.])),
 (10, 20, 0.3): (np.float64(240.0),
  np.float64(36.0),
  array([1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0.])),
 (10, 20, 0.4): (np.float64(240.0),
  np.float64(41.0),
  array([1., 0.,

In [4]:
import pandas as pd

df = pd.DataFrame.from_dict(results, orient="index")
df

,0,1,2
"(10, 10, 0.1)",240.0,28.0,"[1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ..."
"(10, 10, 0.2)",230.0,40.0,"[1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, ..."
"(10, 10, 0.3)",250.0,35.0,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ..."
"(10, 10, 0.4)",240.0,36.0,"[1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
"(10, 10, 0.5)",250.0,43.0,"[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, ..."
...,...,...,...
"(200, 200, 0.1)",250.0,44.0,"[1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, ..."
"(200, 200, 0.2)",250.0,44.0,"[1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, ..."
"(200, 200, 0.3)",250.0,44.0,"[1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, ..."
"(200, 200, 0.4)",250.0,44.0,"[1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, ..."
